# Face Sonification Prototype

ISSUES:
* often, we have a video recorded with a frame rate and sensor data that has been recorded with another frame rate
* augmentation of the video with custom plotting, e.g. yourself plotting markers instead of having them plotted by OpenFace
    * discuss if and how
    * PyQtGraph: pure-python graphics and GUI library built on PyQt / PySide and numpy
* The **timing problem that OpenFace** has seems to be related with OpenCV. The tracked video sometimes is sligtly shorter than the original one and sometimes it is longer. It seems that OpenCV has some problems working with some of the codecs.
    * ffmpeg -i video.file -r 30 -vcodec ffv1 -acodec pcm_s16le output_name
        * in my case, this does not work
    * ffmpeg -i video.file -r 30 -vcodec ffv1 output_name
        * the audio codec should not be relevant
    * split video frames into images if the previous solution fails
        * ffmpeg
        * we have to encode video info in the name of files

TODO:
* record video demos

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

import panson as ps
from panson import bundle

from math import pi, exp, log2

## Container setup and video processing

The following section assumes that OpenFace was installed using docker.

As first step, run the container with the following command:
* `docker run -it --rm --name openface --mount type=bind,source=/home/michele/Desktop/Thesis/media/files,target=/home/openface-build/files algebr/openface:latest`
* Substitute /home/michele/Desktop/Thesis/media/files with the mounting point
* The current working directory must contain a directory **files/**
    * This directory is bound with the --mount option to the files/ directory present in the docker container
    * This directory is shared between the file system of the host and the one of the container

Now we can launch the OpenFace executables (present in the container) from outside the container environment with a command similar to the following:
* `docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -f files/video.avi`
* From python (this notebook) we can call the same process and read the results
* Create the directory files/processed in advance

In [ ]:
!xhost +

In [ ]:
# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')


def feature_extraction_offline(video_name):
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

def feature_extraction_online(pipe='files/pipe'):
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', '0', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def kill_feature_extraction_online():
    # !docker exec -it openface pkill FeatureExt
    command = ['docker', 'exec', CONTAINER_NAME, 'pkill', 'FeatureExt']
    subprocess.run(command)

def read_openface_csv(csv_path):
    return pd.read_csv(csv_path, sep=r',\s*', engine='python')

In [ ]:
def ffmpeg_convert(in_file, out_file):
    
    # this was the original command that appeared in the notebook
    # ffmpeg -y -i "files/processed/phone.avi" -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 128k "files/phone-processed.mp4"
    command = [
        'ffmpeg', '-y',
        '-i', in_file,
        '-c:v', 'libx264',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-c:a', 'libvo_aacenc',
        '-b:a', '128k',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

def ffmpeg_merge(video_file, audio_file, out_file):
    
    # ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y
                
    command = [
        'ffmpeg', '-y',
        '-i', video_file,
        '-i', audio_file,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

In [ ]:
video_name = 'full-rigth-codecs.avi'

In [ ]:
%time feature_extraction_offline(video_name)

In [ ]:
ffmpeg_convert("files/processed/phone.avi", "files/phone-processed.mp4")

## Sonifications

### Setup

#### Supercollider

Here we are using the current **develop** branch of sc3nb.

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

If this does not work, open QJackCtl and link the nodes in the graph manually.

In [ ]:
sc.exit()

In [ ]:
# test sound output
sc.server.blip()

In [ ]:
sc.server.latency = 0.1

#### Panson

##### Offline

In [ ]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

#### Online

In [ ]:
features = None
avg_features = df.filter(regex='(AU.{2}_r)|(pose_R.)|(gaze_angle_.)').columns.to_list()
non_avg_features = None

def init_moving_average():
    global window, window_size
    
    window = pd.DataFrame()
    window_size = 5
    
def moving_average(series):
    global window
    global features
    global non_avg_features
    
    if features is None:
        features = series.index.to_list()
        non_avg_features = list(set(features) - set(avg_features))
    
    window = window.append(series[avg_features])
    
    if window.shape[0] > window_size:
        window = window.drop(window.index[0])
        
    mean_series = window.mean()
    
    # fill other features
    for feature in non_avg_features:
        mean_series[feature] = series[feature]
    
    return mean_series

In [ ]:
import csv

FIFO = os.path.join(FILE_DIR, 'pipe.csv')

def data_generator(path=FIFO):
    with open(path, 'r') as fifo:
        # the reader attempts to execute fifo.readline()
        # which blocks if there are no lines
        reader = csv.reader(fifo, skipinitialspace=True)
        
        header = next(reader)
        
        # init_moving_average()
        
        # the loop ends when the pipe is closed from the writing side
        for i, row in enumerate(reader):
            series = pd.Series(row, header, dtype='float', name=i)
            # series = moving_average(series)
            yield series

### Sonification: AU04 Test
This is a very simple sonification of AU04 (Brow Lowerer). The intensity of AU04 is used here to modulate both the amplitude and the frequency of a continuous synth. As continuous synth, the default synth of sc3nb s2 is used (we will have to instruct the server to load it).

* The intensity range \[0,1\] is mapped into the amplitude range \[0,0.3\], where 0.3 will be the maximum amplitude of the sound. The sonification has a parameter amp that can be used to scale this range.
* The intensity range \[0,5\] is mapped into the midi range \[69,81\]

In [ ]:
# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    def __init__(self):
        super().__init__()
        self.amp = 1
    
    @bundle
    def init(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        # lag time is decided based on the frame rate
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})

    @bundle
    def _process(self, row):
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
son = AU04ContinuousSonification()
son

#### Offline

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU04_r', 'AU12_r'], queue_size=50)
dp = ps.DataPlayer(son, feature_display=feature_display).load(df)

# feature_display.show(fps=30)
display(son)
display(dp)

In [ ]:
dp.rate = 1

In [ ]:
dp.export("score.wav", header_format="WAV")

#### Online

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU04_r', 'AU12_r'], 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(fps=20)
display(son)
rtdp

### Sonification: Pentatonic eyes + brows
* 1: Inner Brow Raiser
* 2: Outer Brow Raiser (unilateral)
* 4: Brow Lowerer
* 5: Upper Lid Raiser
* 6: Cheek Raiser
* 7: Lid Tightener

In [ ]:
class PentatonicContinuous(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    BASE_TONE = 69
    PENTATONIC = [
        scn.midicps(BASE_TONE),
        scn.midicps(BASE_TONE + 3),
        scn.midicps(BASE_TONE + 5),
        scn.midicps(BASE_TONE + 7),
        scn.midicps(BASE_TONE + 10),
        scn.midicps(BASE_TONE + 12)
    ]
    
    def __init__(self):
        super().__init__()
        
        # parameters default
        self.amp = 1
    
    @bundle
    def init(self):
        # load default synth s2
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.au01_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[5], "lg": 0.015})
        self.au02_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[4], "lg": 0.015})
        self.au04_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[3], "lg": 0.015})
        self.au05_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[2], "lg": 0.015})
        self.au06_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[0], "lg": 0.015})
        self.au07_synth = scn.Synth("s2", {"amp": 0, "freq": self.PENTATONIC[1], "lg": 0.015})
    
    @bundle
    def _process(self, row):
        intensity = row["AU01_r"]
        amp = self.map_intensity(intensity)
        self.au01_synth.set("amp", self.amp * amp)
        
        intensity = row["AU02_r"]
        amp = self.map_intensity(intensity)
        self.au02_synth.set("amp", self.amp * amp)
        
        intensity = row["AU04_r"]
        amp = self.map_intensity(intensity)
        self.au04_synth.set("amp", self.amp * amp)
        
        intensity = row["AU05_r"]
        amp = self.map_intensity(intensity)
        self.au05_synth.set("amp", self.amp * amp)
        
        intensity = row["AU06_r"]
        amp = self.map_intensity(intensity)
        self.au06_synth.set("amp", self.amp * amp)
        
        intensity = row["AU07_r"]
        amp = self.map_intensity(intensity)
        self.au07_synth.set("amp", self.amp * amp)
        
    @staticmethod
    def map_intensity(intensity):
        if intensity < 1.0:
            amp = 0
        else:
            db = scn.linlin(intensity, 1, 5, -20, -5, "minmax")
            amp = scn.dbamp(db)

        return amp

In [ ]:
son = PentatonicContinuous()
son

In [ ]:
interesting_features = ["AU01_r", "AU02_r", "AU04_r", "AU05_r", "AU06_r", "AU07_r"]
feature_display = ps.LiveFeatureDisplay(interesting_features, 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(fps=10)
display(son)
display(rtdp)

### Sonification: Pentatonic eyes + brows with MdaPiano

In [ ]:
class PentatonicMda(ps.Sonification):
    
    BASE_TONE = 69
    PENTATONIC = [
        scn.midicps(BASE_TONE),
        scn.midicps(BASE_TONE + 3),
        scn.midicps(BASE_TONE + 5),
        scn.midicps(BASE_TONE + 7),
        scn.midicps(BASE_TONE + 10),
        scn.midicps(BASE_TONE + 12)
    ]
    
    piano_def = scn.SynthDef(
        "mdapiano",
        r"""{ |freq=440, gate=1, vel=100, amp=1|
            var piano = MdaPiano.ar(
                freq,
                gate,
                vel,
                decay: 0,
                release: 0,
                hard: 0,
                stereo: 0,
                sustain: 0,
                mul: amp
            );
            DetectSilence.ar(piano, 0.01, doneAction:2);
            Out.ar(0, piano);
        }"""
    )
    
    @bundle
    def init(self):
        self.piano_def.add()

    @bundle
    def start(self):
        self.old_range_level_01 = 0
        self.old_range_level_02 = 0
        self.old_range_level_04 = 0
        self.old_range_level_05 = 0
        self.old_range_level_06 = 0
        self.old_range_level_07 = 0
        self.old_range_level_09 = 0
        self.old_range_level_10 = 0
        self.old_range_level_12 = 0
        self.old_range_level_14 = 0
        self.old_range_level_15 = 0
        self.old_range_level_17 = 0
        self.old_range_level_20 = 0
        self.old_range_level_23 = 0
        self.old_range_level_25 = 0
        self.old_range_level_26 = 0
    
    @bundle
    def stop(self):
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU01_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_01, cur_range_level, self.PENTATONIC[5])
        self.old_range_level_01 = cur_range_level
        
        intensity = row["AU02_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_02, cur_range_level, self.PENTATONIC[4])
        self.old_range_level_02 = cur_range_level
        
        intensity = row["AU04_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_04, cur_range_level, self.PENTATONIC[3])
        self.old_range_level_04 = cur_range_level
        
        intensity = row["AU05_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_05, cur_range_level, self.PENTATONIC[2])
        self.old_range_level_05 = cur_range_level
        
        intensity = row["AU06_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_06, cur_range_level, self.PENTATONIC[0])
        self.old_range_level_06 = cur_range_level
        
        intensity = row["AU07_r"]
        cur_range_level = int(intensity)
        self.map_range(self.old_range_level_07, cur_range_level, self.PENTATONIC[1])
        self.old_range_level_07 = cur_range_level        
        
    @staticmethod
    def map_range(old_range_level, cur_range_level, freq):
        if cur_range_level != old_range_level and cur_range_level >= 1:
            vel = scn.linlin(cur_range_level, 1, 5, 40, 127)
            scn.Synth('mdapiano', {"freq": freq, "vel": vel})

In [ ]:
son = PentatonicMda()
son

TODO: adjust amplitude

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
rtdp

### Sonification: Drop blink

In [ ]:
class DropBlink(ps.Sonification):
    
    # hysteresis boundaries
    bounds = ps.FloatRangeSliderParameter(0, 5, 0.1)
    
    # drop definition
    drop_def = scn.SynthDef(
        "drop",
        r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
            var sig, env, fch;
            fch = XLine.kr(freq, freq+dp, dur);
            sig = SinOsc.ar(fch);
            env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
            Out.ar(0, Pan2.ar(sig, pan, env*amp))
        }"""
    )
    
    def __init__(self):
        super().__init__()
        self.bounds = [1, 1.6]
    
    @bundle
    def init(self):
        self.drop_def.add()

    @bundle
    def start(self):
        self.blinking = False
    
    @bundle
    def stop(self):
        # drops die out alone
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")

In [ ]:
son = DropBlink()
son

In [ ]:
feature_display = ps.LiveFeatureDisplay(["AU45_r"], 80)

rtdp = ps.RTDataPlayer(data_generator, son, feature_display=feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)

In [ ]:
feature_display.show(10)
display(son)
rtdp

In [ ]:
class DoubleDropBlink(DropBlink):
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
                scn.Synth("drop", {"freq": 900})
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")

### Multi-percussion (event-based)

Similar samples
* snares
<!--     * <audio controls src="samples/132417__sajmund__percussion-clave-like-hit.wav"/> -->
<!--     * <audio controls src="samples/277397__earmark-audio__efev1-percussion-snare.wav"/> -->
* high bells
<!--     * <audio controls src="samples/339808__inspectorj__hand-bells-c-db-single.wav"/> -->
<!--     * <audio controls src="samples/360327__inspectorj__triangle-8-hard-hit-a.wav"/> -->
<!--     * <audio controls src="samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"/> -->
<!--     * <audio controls src="samples/387715__jagadamba__gong-percussion.wav"/> -->
<!--     * <audio controls src="samples/414563__pjcohen__agogo-bell-low-velocity11.wav"/> -->
<!--     * <audio controls src="samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"/> -->
<!--     * <audio controls src="samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"/> -->
* crash
<!--     * <audio controls src="samples/public domain/209874__veiler__pff-chrash-14.wav"/> -->
* electronic
<!--     * <audio controls src="samples/public domain/487662__phonosupf__electronic-percussion-5.wav"/> -->
<!--     * <audio controls src="samples/207919__altemark__space-snare.wav"/> -->
<!--     * <audio controls src="samples/577014__nezuai__cartoon-percussion-3.wav"/> -->
* drums
<!--     * <audio controls src="samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"/> -->
    * <audio controls src="samples/public domain/439828__twentytwentymusic__electronic-percussion-6.wav"/>
<!--     * <audio controls src="samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"/> -->
* bass drum
<!--     * <audio controls src="samples/138358__minorr__bass-drum-p.wav"/> -->
<!--     * <audio controls src="samples/234746__sonidotv__legno-10.wav"/> -->

Concepts:
* Sounds that usually plays together should be distinguishable
* Sounds that play more often should be less intrusive
* One area should have sounds that are somehow related

<hr>

* 1: Inner Brow Raiser - <audio controls src="samples/modified/au01.wav"/>
* 2: Outer Brow Raiser (unilateral) - <audio controls src="samples/modified/au02.wav"/>
* 4: Brow Lowerer - <audio controls src="samples/modified/au04.wav"/>

<hr>

* 5: Upper Lid Raiser - <audio controls src="samples/modified/au05.wav"/>
* 6: Cheek Raiser - <audio controls src="samples/modified/au06.wav"/>
* 7: Lid Tightener - <audio controls src="samples/modified/au07.wav"/>

<hr>

* 9: Nose Wrinkler (usually goes along with 4 and 10) - <audio controls src="samples/modified/au09.wav"/>
* 10: Upper Lip Raiser - <audio controls src="samples/modified/au10.wav"/>

<hr>

* 12: Lip Corner Puller - <audio controls src="samples/modified/au12.wav"/>
* 14: Dimpler - <audio controls src="samples/modified/au14.wav"/>
* 15: Lip Corner Depressor - <audio controls src="samples/modified/au15.wav"/>
* 20: Lip Stretcher - <audio controls src="samples/modified/au20.wav"/>

<hr>

* 23: Lip Tightener - <audio controls src="samples/modified/au23.wav"/>

<hr>

* 17: Chin Raiser - <audio controls src="samples/modified/au17.wav"/>
* 25: Lips Part (relax Mentalis, antagonist of AU17) - <audio controls src="samples/modified/au25.wav"/>
* 26: Jaw Drop (usually goes along with 25) - <audio controls src="samples/modified/au26.wav"/>

<hr>

* 28: Lip Suck (usually along with 26) - <audio controls src="samples/modified/au28.wav"/>
    * OpenFace only provides presence information

<hr>

* 45: Blink (drop)

In [ ]:
AU01_SAMPLE_PATH = "samples/modified/au01.wav"
AU02_SAMPLE_PATH = "samples/modified/au02.wav"
AU04_SAMPLE_PATH = "samples/modified/au04.wav"
AU05_SAMPLE_PATH = "samples/modified/au05.wav"
AU06_SAMPLE_PATH = "samples/modified/au06.wav"
AU07_SAMPLE_PATH = "samples/modified/au07.wav"
AU09_SAMPLE_PATH = "samples/modified/au09.wav"
AU10_SAMPLE_PATH = "samples/modified/au10.wav"
AU12_SAMPLE_PATH = "samples/modified/au12.wav"
AU14_SAMPLE_PATH = "samples/modified/au14.wav"
AU15_SAMPLE_PATH = "samples/modified/au15.wav"
AU17_SAMPLE_PATH = "samples/modified/au17.wav"
AU20_SAMPLE_PATH = "samples/modified/au20.wav"
AU23_SAMPLE_PATH = "samples/modified/au23.wav"
AU25_SAMPLE_PATH = "samples/modified/au25.wav"
AU26_SAMPLE_PATH = "samples/modified/au26.wav"
AU28_SAMPLE_PATH = "samples/modified/au28.wav"

#### Sonification: Percussive

In [ ]:
class Percussive(ps.Sonification):

    playbuf_def = scn.SynthDef(
        "playbuf",
        r"""{| out=0, bufnum=0, rate=1, amp=1, pan=0 |
            var sig;
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )

    @bundle
    def init(self):
        self.playbuf_def.add()
        
        self.buf01 = scn.Buffer().read(AU01_SAMPLE_PATH)
        self.buf02 = scn.Buffer().read(AU02_SAMPLE_PATH)
        self.buf04 = scn.Buffer().read(AU04_SAMPLE_PATH)
        self.buf05 = scn.Buffer().read(AU05_SAMPLE_PATH)
        self.buf06 = scn.Buffer().read(AU06_SAMPLE_PATH)
        self.buf07 = scn.Buffer().read(AU07_SAMPLE_PATH)
        self.buf09 = scn.Buffer().read(AU09_SAMPLE_PATH)
        self.buf10 = scn.Buffer().read(AU10_SAMPLE_PATH)
        self.buf12 = scn.Buffer().read(AU12_SAMPLE_PATH)
        self.buf14 = scn.Buffer().read(AU14_SAMPLE_PATH)
        self.buf15 = scn.Buffer().read(AU15_SAMPLE_PATH)
        self.buf17 = scn.Buffer().read(AU17_SAMPLE_PATH)
        self.buf20 = scn.Buffer().read(AU20_SAMPLE_PATH)
        self.buf23 = scn.Buffer().read(AU23_SAMPLE_PATH)
        self.buf25 = scn.Buffer().read(AU25_SAMPLE_PATH)
        self.buf26 = scn.Buffer().read(AU26_SAMPLE_PATH)
        # self.buf28 = scn.Buffer().read(AU28_SAMPLE_PATH)

    @bundle
    def start(self):        
        self.old_range_level_01 = 0
        self.old_range_level_02 = 0
        self.old_range_level_04 = 0
        self.old_range_level_05 = 0
        self.old_range_level_06 = 0
        self.old_range_level_07 = 0
        self.old_range_level_09 = 0
        self.old_range_level_10 = 0
        self.old_range_level_12 = 0
        self.old_range_level_14 = 0
        self.old_range_level_15 = 0
        self.old_range_level_17 = 0
        self.old_range_level_20 = 0
        self.old_range_level_23 = 0
        self.old_range_level_25 = 0
        self.old_range_level_26 = 0
    
    @bundle
    def stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        # cast intensity to integer
        cur_range_level_01 = int(row["AU01_r"])
        self.map_range(self.old_range_level_01, cur_range_level_01, self.buf01)
        # update old_range_level
        self.old_range_level_01 = cur_range_level_01
        
        cur_range_level_02 = int(row["AU02_r"])
        self.map_range(self.old_range_level_02, cur_range_level_02, self.buf02)
        self.old_range_level_02 = cur_range_level_02
        
        cur_range_level_04 = int(row["AU04_r"])
        self.map_range(self.old_range_level_04, cur_range_level_04, self.buf04)
        self.old_range_level_04 = cur_range_level_04
        
        cur_range_level_05 = int(row["AU05_r"])
        self.map_range(self.old_range_level_05, cur_range_level_05, self.buf05)
        self.old_range_level_05 = cur_range_level_05
        
        cur_range_level_06 = int(row["AU06_r"])
        self.map_range(self.old_range_level_06, cur_range_level_06, self.buf06)
        self.old_range_level_06 = cur_range_level_06
        
        cur_range_level_07 = int(row["AU07_r"])
        self.map_range(self.old_range_level_07, cur_range_level_07, self.buf07)
        self.old_range_level_07 = cur_range_level_07
        
        cur_range_level_09 = int(row["AU09_r"])
        self.map_range(self.old_range_level_09, cur_range_level_09, self.buf09)
        self.old_range_level_09 = cur_range_level_09
        
        cur_range_level_10 = int(row["AU10_r"])
        self.map_range(self.old_range_level_10, cur_range_level_10, self.buf10)
        self.old_range_level_10 = cur_range_level_10
        
        cur_range_level_12 = int(row["AU12_r"])
        self.map_range(self.old_range_level_12, cur_range_level_12, self.buf12)
        self.old_range_level_12 = cur_range_level_12
        
        cur_range_level_14 = int(row["AU14_r"])
        self.map_range(self.old_range_level_14, cur_range_level_14, self.buf14)
        self.old_range_level_14 = cur_range_level_14
        
        cur_range_level_15 = int(row["AU15_r"])
        self.map_range(self.old_range_level_15, cur_range_level_15, self.buf15)
        self.old_range_level_15 = cur_range_level_15
        
        cur_range_level_17 = int(row["AU17_r"])
        self.map_range(self.old_range_level_17, cur_range_level_17, self.buf17)
        self.old_range_level_17 = cur_range_level_17        
        
        cur_range_level_20 = int(row["AU20_r"])
        self.map_range(self.old_range_level_20, cur_range_level_20, self.buf20)
        self.old_range_level_20 = cur_range_level_20
        
        cur_range_level_23 = int(row["AU23_r"])
        self.map_range(self.old_range_level_23, cur_range_level_23, self.buf23)
        self.old_range_level_23 = cur_range_level_23
        
        cur_range_level_25 = int(row["AU25_r"])
        self.map_range(self.old_range_level_25, cur_range_level_25, self.buf25)
        self.old_range_level_25 = cur_range_level_25
        
        cur_range_level_26 = int(row["AU26_r"])
        self.map_range(self.old_range_level_26, cur_range_level_26, self.buf26)
        self.old_range_level_26 = cur_range_level_26

    @staticmethod
    def map_range(old_range_level, cur_range_level, buf):
        if cur_range_level != old_range_level and cur_range_level >= 1:
            db = scn.linlin(cur_range_level, 1, 5, -40, 0, "minmax")
            scn.Synth("playbuf", {"bufnum": buf.bufnum, "amp": scn.dbamp(db)})

In [ ]:
son = Percussive()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
rtdp

#### Sonification: Multi-percussion sound with intensity direction

In [ ]:
class DirectionalPercussive(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    pan = ps.CheckboxParameter()
    # hysteresis bounds relative to each intensity level
    bounds = ps.FloatRangeSliderParameter(-1, +1)

    playbuf_def = scn.SynthDef(
        "playbuf_bend",
        r"""{| out=0, bufnum=0, rateInitial=1, amp=1, pan=0, breakTime, rateFinal |
            var sig, rate;

            var rateAvg = (rateInitial + rateFinal) / 2;
            var sampleRateAvg = rateAvg * BufSampleRate.kr(bufnum);
            var breakFrame = breakTime * BufSampleRate.kr(bufnum);
            // mono signal: frames = samples
            var remainingFrames = BufSamples.kr(bufnum) - breakFrame;
            // calculate remaining time with dynamic rate
            var remainingTime = remainingFrames / sampleRateAvg;

            rate = EnvGen.kr(
                Env(
                    [rateInitial, rateInitial, rateFinal],
                    [breakTime, remainingTime]
                )
            );
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )
    
    AUs = {
        1: {
            'label': "AU01_r",
            'path': AU01_SAMPLE_PATH,
            'break_time': 0.2,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        2: {
            'label': "AU02_r",
            'path': AU02_SAMPLE_PATH,
            'break_time': 0.2,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        4: {
            'label': "AU04_r",
            'path': AU04_SAMPLE_PATH,
            'break_time': 0.2,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        5: {
            'label': "AU05_r",
            'path': AU05_SAMPLE_PATH,
            'break_time': 0.05,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        6: {
            'label': "AU06_r",
            'path': AU06_SAMPLE_PATH,
            'break_time': 0.03,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        7: {
            'label': "AU07_r",
            'path': AU07_SAMPLE_PATH,
            'break_time': 0.1,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        9: {
            'label': "AU09_r",
            'path': AU09_SAMPLE_PATH,
            'break_time': 0.2,
            'rate_up': 1.5,
            'rate_down': 0.7
        },
        10: {
            'label': "AU10_r",
            'path': AU10_SAMPLE_PATH,
            'break_time': 0.15,
            'rate_up': 1.2,
            'rate_down': 0.9
        },
        12: {
            'label': "AU12_r",
            'path': AU12_SAMPLE_PATH,
            'break_time': 0.1,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        14: {
            'label': "AU14_r",
            'path': AU14_SAMPLE_PATH,
            'break_time': 0.1,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        15: {
            'label': "AU15_r",
            'path': AU15_SAMPLE_PATH,
            'break_time': 0.05,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        17: {
            'label': "AU17_r",
            'path': AU17_SAMPLE_PATH,
            'break_time': 0.05,
            'rate_up': 1.5,
            'rate_down': 0.8
        },
        20: {
            'label': "AU20_r",
            'path': AU20_SAMPLE_PATH,
            'break_time': 0.1,
            'rate_up': 1.1,
            'rate_down': 0.9
        },
        23: {
            'label': "AU23_r",
            'path': AU23_SAMPLE_PATH,
            'break_time': 0.15,
            'rate_up': 1.25,
            'rate_down': 0.85
        },
        25: {
            'label': "AU25_r",
            'path': AU25_SAMPLE_PATH,
            'break_time': 0.005,
            'rate_up': 2,
            'rate_down': 0.5
        },
        26: {
            'label': "AU26_r",
            'path': AU26_SAMPLE_PATH,
            'break_time': 0.05,
            'rate_up': 2,
            'rate_down': 0.5
        }
    }
    
    def __init__(self, pan=False):
        super().__init__()
        self.amp = 0.3
        self.pan = False
        self.bounds = [-0.3, +0.3]

        self.old_levels = {}

    @bundle
    def init(self):
        self.playbuf_def.add()
        
        self.buffers = {}
        
        # allocate buffers
        for i, info in self.AUs.items():
            self.buffers[i] = scn.Buffer().read(info['path'])

    @bundle
    def start(self):
        # initialize old range levels
        for i in self.AUs.keys():
            self.old_levels[i] = 0
    
    @bundle
    def stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        for i, info in self.AUs.items():
            intensity = row[info['label']]
            cur_range_level = self.map_intensity(intensity, self.old_levels[i])
            
            if cur_range_level != self.old_levels[i] and cur_range_level >= 1:
                db = scn.linlin(cur_range_level, 1, 5, -40, 0, "minmax")
                amp = scn.dbamp(db)
            
                if cur_range_level > self.old_levels[i]:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[i].bufnum,
                            "amp": self.amp * amp,
                            "pan": 1 if self.pan else 0,
                            "breakTime": info['break_time'],
                            "rateFinal": info['rate_up']
                        }
                    )
                else:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[i].bufnum,
                            "amp": self.amp * amp,
                            "pan": -1 if self.pan else 0,
                            "breakTime": info['break_time'],
                            "rateFinal": info['rate_down']
                        }
                    )
                    
            # update old_range_level
            self.old_levels[i] = cur_range_level

    def map_intensity(self, intensity, old_level):
        cur_level = int(intensity)
        
        if cur_level == old_level:
            return cur_level        
        elif cur_level > old_level:
            return cur_level if intensity > cur_level + self.bounds[1] else old_level
        elif cur_level < old_level:
            return cur_level if intensity < old_level + self.bounds[0] else old_level
            
    def free(self):
        # deallocate buffers
        for info in self.AU.values():
            info['buf'].free()

In [ ]:
son = DirectionalPercussive()
son

In [ ]:
son.free()

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
rtdp

### Head

In [ ]:
feature_display = ps.LiveFeatureDisplay(['pose_Rx', 'pose_Ry', 'pose_Rz'], queue_size=100)

#### Sonification: Head rotations with constant amplitude

In [ ]:
class Head(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    def __init__(self):
        super().__init__()
        self.amp = 0.1
    
    @bundle
    def init(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        pitch = scn.linlin(row['pose_Rx'], -pi/2, +pi/2, 69+12, 69-12)
        pan =   scn.linlin(row['pose_Ry'], -pi/2, +pi/2, +1, -1)
        num =   scn.linlin(row['pose_Rz'], -pi/4, +pi/4, 1, 7)
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "num", num
        )

In [ ]:
son = Head()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

#### Sonification: Head rotations with silent neutral position

The amplitude is mapped linearly based on the biggest rotation.
* a linear mapping makes quiet sounds more distinguishable
* every axe has a silence thrashold, so that a head in a pretty neutral position does not generate any sound

In [ ]:
class SilentHead(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    # max expected values for each rotation
    rx_bound = pi/2
    ry_bound = pi/2
    rz_bound = pi/4
    
    # in radians
    rx_silence_thrashold = rx_bound / 10
    ry_silence_thrashold = ry_bound / 10
    rz_silence_thrashold = rz_bound / 10
    
    def __init__(self):
        super().__init__()
        self.amp = 0.3
    
    @bundle
    def init(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        # linear mapping
        amp = max(
            # clips values under silence thrashold
            scn.linlin(abs(rx), self.rx_silence_thrashold, +self.rx_bound, 0, 1, "minmax"),
            scn.linlin(abs(ry), self.ry_silence_thrashold, +self.ry_bound, 0, 1, "minmax"),
            scn.linlin(abs(rz), self.rx_silence_thrashold, +self.rz_bound, 0, 1, "minmax")
        )
        
        pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, 69+12, 69-12)
        pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1)
        num =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 7)
        
        self.synth.set(
            "amp", self.amp * amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "num", num
        )

In [ ]:
son = SilentHead()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

#### Sonification: Noisy head rotations

In [ ]:
class NoisyHead(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    log_mapping = ps.CheckboxParameter()
    
    # max expected values for each rotation
    rx_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    ry_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    rz_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    
    synth_def = scn.SynthDef(
        "bpf_noise",
        r"""{ | amp=1, pan=0, lg=0.5, freq=440, rq=0.2 |
            var sig;
            sig = PinkNoise.ar(amp);
            sig = BPF.ar(
                sig,
                freq.lag(lg),
                rq.lag(lg),
                // when a bandpass filter narrows, the amplitude decreases: this will balance it
                1/rq.sqrt.lag(lg)
            );
            Out.ar(0, Pan2.ar(sig, pan));
        }"""
    )
    
    def __init__(self):
        super().__init__()
        self.amp = 0.3
        self.base_tone = 69
        self.log_mapping = False
        
        self.rx_bound = pi/2
        self.ry_bound = pi/2
        self.rz_bound = pi/4
    
    @bundle
    def init(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("bpf_noise", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        # use log() - 1 to map
        if self.log_mapping:
            rx_log = log2(scn.linlin(abs(rx), 0, self.rx_bound, 1, 2, 'minmax'))
            rx_midi = scn.linlin(rx_log, 0, 1, 0, 12, 'minmax')
            sign = 1 if rx >= 0 else -1
            pitch = self.base_tone + sign * rx_midi
            
            ry_exp = scn.linlin(abs(ry), 0, self.ry_bound, 1, 2, 'minmax')
            sign = 1 if ry >= 0 else -1
            pan = sign * log2(ry_exp)
                        
            q =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 100, 'minmax')
            
        else:
            pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, 69+12, 69-12, "minmax")
            pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1, "minmax")
            # linexp mapping to quality
            q =   exp(scn.linlin(rz, -self.rz_bound, +self.rz_bound, log2(1), log2(100), 'minmax'))
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "rq", 1/q
        )

In [ ]:
son = NoisyHead()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

### Gaze

**gaze_N_x, gaze_N_y, gaze_N_z**: Eye gaze **direction vector** in world coordinates for eye left and right eye (normalized)
- N = 0: leftmost eye in the image
- N = 1: rightmost eye in the image

**gaze_angle_x, gaze_angle_y**: Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more **easy to use format** than gaze vectors.
- If a person is looking left-right this will results in the change of gaze_angle_x (from positive to negative)
- if a person is looking up-down this will result in change of gaze_angle_y (from negative to positive)
- if a person is looking straight ahead both of the angles will be close to 0 (within measurement error).

In [ ]:
feature_display = ps.LiveFeatureDisplay(['gaze_angle_x', 'gaze_angle_y'], queue_size=100)

#### Sonification: Gaze with constant amplitude

In [ ]:
class Gaze(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    def __init__(self):
        super().__init__()
        self.amp = 0.1
    
    @bundle
    def init(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        pitch = scn.linlin(row['gaze_angle_y'], -pi/2, +pi/2, 69+12, 69-12)
        pan =   scn.linlin(row['gaze_angle_x'], -pi/2, +pi/2, +1, -1)
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan
        )

In [ ]:
son = Gaze()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

#### Sonification: Gaze with silent neutral position

In [ ]:
class SilentGaze(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    gx_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    gy_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    
    freq = ps.FreqSliderParameter()
    
    def __init__(self):
        super().__init__()
        
        self.amp = 0.1
        self.gx_silence_thrashold = pi/2 / 10
        self.gy_silence_thrashold = pi/2 / 10
        
        self.freq = 50
    
    @bundle
    def init(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        gx, gy = row[['gaze_angle_x', 'gaze_angle_y']]
        
        # linear mapping
        amp = max(
            # clips values under silence thrashold
            scn.linlin(abs(gx), self.gx_silence_thrashold, pi/2, 0, 1, "minmax"),
            scn.linlin(abs(gy), self.gy_silence_thrashold, pi/2, 0, 1, "minmax")
        )
        
        pan =   scn.linlin(gx, -pi/2, +pi/2, -1, +1)
        pitch = scn.linlin(gy, -pi/2, +pi/2, 69+12, 69-12)
        
        self.synth.set(
            "amp", self.amp * amp,
            "freq", scn.midicps(pitch),
            "pan", pan
        )

In [ ]:
son = SilentGaze()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

### Smile

AUs regarding the smile recognised by OpenFace:
* AU06 - Cheek raiser
* AU12 - Lip Corner Puller
* AU14 - Dimpler
* AU15 - Lip Corner Depressor
* AU17 - Chin Raiser

Expected to go together:
* AU06 - AU12
* AU15 - AU17

For now we will ignore AU14 for simplicity, and we will sonify it using the percussive approach.

TODO: musical sonification of the smile

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU06_r', 'AU12_r', 'AU15_r', 'AU17_r'], queue_size=100)

In [ ]:
class DustSmile(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    
    synth_def = scn.SynthDef(
        "discrete_rev",
        r"""{ | amp=1, freq=440, density=2, mix=0.5, room=0.5, damp=0.2, lg=0.1 |
            var trig, sig, env;
            sig = SinOsc.ar(freq);
            // transform signal into short blips
            trig = Dust.kr(density);
            env = EnvGen.kr(Env.perc(0.001, 0.05), trig);
            sig = sig * env;
            sig = FreeVerb.ar(sig, mix.lag(lg), room.lag(lg), damp.lag(lg), amp.lag(lg));
            Out.ar(0, sig!2);
        }"""
    )
    
    def __init__(self):
        super().__init__()
        self.amp = 0.3
        self.base_tone = 69
    
    @bundle
    def init(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("discrete_rev", {"amp": 0})
    
    @bundle
    def _process(self, row):
        # intensities
        au06, au12, au15, au17 = row[['AU06_r', 'AU12_r', 'AU15_r', 'AU17_r']]

        if au12 > 1 and au15 > 1:
            # this should not happen usually
            return
        
        if au12 > 1:
            pitch = scn.linlin(au12, 1, 5, self.base_tone, self.base_tone+12, 'minmax')
            mix = scn.linlin(au06, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au12, au06), 1, 5, 5, 30, 'minmax')
        elif au15 > 1:
            pitch = scn.linlin(au15, 1, 5, self.base_tone, self.base_tone-12, 'minmax')
            mix = scn.linlin(au17, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au15, au17), 1, 5, 5, 30, 'minmax')
        else:
            pitch = 69
            mix = 0
            density = 0
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "mix", mix,
            "density", density,
        )

In [ ]:
son = DustSmile()
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son, feature_display)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
feature_display.show(5)
rtdp

### All togheter...

In [ ]:
son = ps.GroupSonification([DirectionalPercussive(), DropBlink(), NoisyHead(), SilentGaze()])
son

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son)\
        .add_listen_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)
rtdp